### IMPORTS

In [1]:
import json
import subprocess
import sys
import time

In [2]:
import hashlib
import base64

In [3]:
from web3 import Web3

### FIX FOR WINDOWS ENVIRONMENT

In [4]:
import pkg_resources.py2_warn

### LOGGING BASELINE

In [5]:
def log(msg):
    print('[LAUNCHER] ' + msg)

### LOAD DATA FROM EXTERNAL FILES

In [6]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [7]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

In [8]:
with open('config/latest.json') as json_file:
    latest = json.load(json_file)

In [9]:
log('FETCHED DATA FROM EXTERNAL FILES')

[LAUNCHER] FETCHED DATA FROM EXTERNAL FILES


### GATEWAY CONNECTION CHECK

In [10]:
def connection(protocol):
    
    # GATEWAY TO PROTOCOL
    gateway = Web3(Web3.WebsocketProvider(
        'ws://' + settings['gateways'][protocol]['host'] + ':' + str(settings['gateways'][protocol]['port'])
    ))
    
    # IF CONNECTION IS ESTABLISHED
    if gateway.isConnected():
        
        # SHOW MESSAGE
        message = protocol + ' GATEWAY CONNECTION ESTABLISHED'
        log(message.upper())
        
        # FINALLY RETURN CONNECTION STRING
        return gateway
        
    # OTHERWISE, SHOW ERROR & DIE
    else:
        log('COULD NOT CONNECT TO GATEWAY. SHUTTING DOWN.')
        exit

### ATTEMPT TO CONNECT

In [11]:
blockchain = connection('blockchain')

[LAUNCHER] BLOCKCHAIN GATEWAY CONNECTION ESTABLISHED


In [12]:
whisper = connection('whisper')

[LAUNCHER] WHISPER GATEWAY CONNECTION ESTABLISHED


### EXTRACT THE SHH API

In [13]:
shh = whisper.geth.shh

### GENERATE WHISPER ID

In [14]:
whisper_id = shh.newKeyPair()

In [15]:
log('WHISPER ID GENERATED')

[LAUNCHER] WHISPER ID GENERATED


### HASH DEVICE IDENTIFIER

In [16]:
def hash_id(data):
    
    # REMOVE WHITESPACES
    to_string = json.dumps(data, sort_keys=False, indent=2)
    
    # ENCODE THE STRING WITH UTF8
    encoded = to_string.encode('utf-8')
    
    # HASH ENCODED DATA
    hashed = hashlib.sha256(encoded).hexdigest()
    
    return hashed

### DEVICE OUTLINE

In [17]:
class create_device():
    def __init__(self):
        self.hash = hash_id(device_info)
    
    # LOCATE & SET DEVICE CONTRACT ADDRESS
    def set_contract(self, contract):
        self.contract = contract
    
    # REDIRECT TO PARENT
    def read(self, details):
        return self.contract.read(details)
    
    # REDIRECT TO PARENT
    def write(self, details):
        return self.contract.write(details)
    
    # REDIRECT TO PARENT
    def event(self, name):
        return self.contract.event(name)

In [18]:
device = create_device()

In [19]:
log('DEVICE ID HASHED')

[LAUNCHER] DEVICE ID HASHED


### UNIFORM SMART CONTRACT OUTLINE

In [20]:
class contract:
    
    # ON LOAD..
    def __init__(self, block):
        
        # CONSTRUCT USABLE CONTRACT
        self.contract = blockchain.eth.contract(
            address = block['address'],
            abi = block['abi']
        )
        
        # SET ADDRESS REFERENCE
        self.address = block['address']
    
    # READ FROM CONTRACT
    def read(self, details):
        
        # WITH PARAMS
        if ('params' in details):
            return self.contract.functions[details['func']](details['params']).call()
        
        # WITHOUT PARAMS
        else:
            return self.contract.functions[details]().call()
    
    # WRITE TO CONTRACT
    def write(self, details):
        try:
            
            # CREATE BASE TRANSACTION
            tx = {
                'from': settings['keys']['public'],
                'to': self.contract.address,
                'data': self.contract.encodeABI(
                    fn_name = details['func'],
                    args = details['params']
                )
            }
            
            # ESTIMATE GAS VALUE & STITCH IN REMAINING PROPS
            tx['gas'] = blockchain.eth.estimateGas(tx)
            tx['gasPrice'] = blockchain.toWei(20, 'gwei')
            tx['nonce'] = blockchain.eth.getTransactionCount(settings['keys']['public'])

            # SIGN TRANSCTION WITH PRIVATE KEY
            signed = blockchain.eth.account.sign_transaction(tx,
                private_key = settings['keys']['private']
            )

            # SEND THE TRANSACTION
            blockchain.eth.sendRawTransaction(signed.rawTransaction)

            # SUCCESS
            return True
        
        # IF THE TRANSACTION IS REVERTED, SHOW ERROR
        except ValueError as error:
            return error
    
    # EVENT FILTER
    def event(self, name):
        return self.contract.events[name].createFilter(fromBlock="latest")

### SERIALIZE MANAGER CONTRACTS

In [21]:
device_manager = contract(latest['devicemanager'])

In [22]:
task_manager = contract(latest['taskmanager'])

In [23]:
log('MANAGER CONTRACTS SERIALIZED')

[LAUNCHER] MANAGER CONTRACTS SERIALIZED


### SERIALIZE DEVICE CONTRACT

In [24]:
temp_contract = contract({
    'address': device_manager.read({
        'func': 'fetch_device',
        'params': device.hash
    }),
    'abi': latest['device']['abi']
})

In [25]:
device.set_contract(temp_contract)

In [26]:
log('DEVICE CONTRACT SERIALIZED')

[LAUNCHER] DEVICE CONTRACT SERIALIZED


### HELPER FUNCS

In [27]:
def filter_backlog(data):
    
    # FILTER ZEROS
    filtered = filter(lambda x: x != '0x0000000000000000000000000000000000000000', data)
    
    # CONVERT TO LIST & RETURN
    return list(filtered)

In [28]:
def encode(data):
    
    # STRINGIFY & CONVERT TO BYTES
    stringified = json.dumps(data)
    to_bytes = str.encode(stringified)
    
    # ENCODE
    encoded = base64.b64encode(to_bytes)
    
    # RETURN AS STRING
    return encoded.decode()

In [29]:
def decode(compressed):
    
    # ATTEMPT TO DECODE & PARSE AS JSON
    try:
        to_bytes = base64.b64decode(compressed)
        return json.loads(to_bytes)
    
    # OTHERWISE, RETURN EMPTY OBJECT
    except:
        return {}

In [30]:
def compare_services(data, base):
    
    # RESULT CONTAINER
    result = []
    
    # CONVERT BASE TO SET FOR QUICKER LOOKUPS
    base = set(base)
    
    # LOOP THROUGH & APPEND CHECK RESULT
    for key in data:
        result.append(key in base)
    
    # FINALLY RETURN
    return result

In [31]:
def compare_discovery(data, base):
    
    # RESULT CONTAINER
    result = []
    
    # LOOP THROUGH DATA KEYS
    for key in data:
        
        # IF THE KEY EXISTS IN THE BASE DICT
        if key in base:
            
            # IF THE VALUE IS SAME IN BOTH DATASET
            if data[key] == base[key]:
                result.append(True)
                
            # OTHERWISE, DEFAULT TO FALSE
            else:
                result.append(False)
                
        # OTHERWISE, DEFAULT TO FALSE
        else:
            result.append(False)
            
    # FINALLY RETURN RESULT
    return result

### GLOBAL TASK BACKLOG

In [32]:
try:
    raw = device.read('details')[1]
except:
    log('THE DEVICE HASH IS NOT REGISTERED')

In [33]:
backlog = filter_backlog(raw)

In [34]:
log('TASKS IN BACKLOG: ' + str(len(backlog)))

[LAUNCHER] TASKS IN BACKLOG: 0


### GLOBAL ACTIVE STATUS 

In [35]:
active = device.read('active')

In [36]:
log('CURRENT ACTIVE STATUS: ' + str(active))

[LAUNCHER] CURRENT ACTIVE STATUS: False


### GLOBAL DISCOVERY STATUS

In [37]:
discoverable = device.read('discoverable')

In [38]:
log('CURRENT DISCOVERABLE STATUS: ' + str(discoverable))

[LAUNCHER] CURRENT DISCOVERABLE STATUS: True


### GLOBAL DISCOVERY PARAMS

In [39]:
compressed = device.read('tags')

In [40]:
discovery_config = decode(compressed)

In [41]:
log('DISCOVERY PARAMS FETCHED')

[LAUNCHER] DISCOVERY PARAMS FETCHED


### GLOBAL SERVICES

In [42]:
services = device.read('fetch_services')

In [43]:
log('SERVICES FETCHED')

[LAUNCHER] SERVICES FETCHED


### EVENT FUNCTIONS

In [44]:
def update_details(event):
    
    # FETCH GLOBAL VARS
    global active
    global discoverable
    global backlog
    global discovery_config
    global services
    
    # EXTRACT RELEVANT VALUES
    latest_active = event['args']['active']
    latest_discoverable = event['args']['discoverable']
    latest_config = decode(event['args']['tags'])
    latest_services = event['args']['services']

    # IF ACTIVE STATUS HAS CHANGED
    if (latest_active != active):
        
        # UPDATE ACTIVE STATUS
        active = latest_active
        
        # SEND MSG
        log('ACTIVE STATUS CHANGED TO: ' + str(latest_active))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_discoverable != discoverable):
        
        # UPDATE ACTIVE STATUS
        discoverable = latest_discoverable
        
        # SEND MSG
        log('DISCOVERABLE STATUS CHANGED TO: ' + str(latest_discoverable))
        
    # IF DISCOVERABLE STATUS HAS CHANGED
    if (latest_config != discovery_config):
        
        # UPDATE ACTIVE STATUS
        discovery_config = latest_config
        
        # SEND MSG
        log('DISCOVERY CONFIG CHANGED')
        
    # IF SERVICES HAVE CHANGED
    if (latest_services != services):
        
        # UPDATE ACTIVE STATUS
        services = latest_services
        
        # SEND MSG
        log('SERVICES HAVE CHANGED')
        
    # UPDATE BACKLOG
    raw_backlog = event['args']['backlog']
    backlog = filter_backlog(raw_backlog)

In [45]:
def update_middleware():
    
    # PRINT REACTION
    log('MIDDLEWARE UPDATE TRIGGERED')

    # TRIGGER UPDATE SCRIPT
    # subprocess.call('./patcher')

    # CLOSE LANCHER
    # sys.exit(0)

In [46]:
def perform_task(task):
    
    # SHOW MSG
    log('STARTING TASK: ' + task)

    # TASK RETURN PARAMS
    ipfs = 'QmWATWQ7fVPP2EFGu71UkfnqhYXDYH566qy47CnJDgvs8u'
    key = '0x4f7a87EE7A53ae8606e80FE96a47038DF8ab7956'

    # SLEEP FOR 5 SECONDS
    time.sleep(5)
    
    # ADD WHATEVER TASK MODULE HERE

    # SUBMIT THE TASK RESULT
    task_manager.write({
        'func': 'complete',
        'params': [task, ipfs, key]
    })

    # SHOW MSG
    log('TASK COMPLETED')

In [47]:
def process_message(event):

    # SERIALIZE EVENT PARAMS
    author = blockchain.toHex(event['sig'])
    payload = blockchain.toText(event['payload'])

    # DECODE THE PAYLOAD
    data = decode(payload)
    
    # REQUIRED KEYS FOR VALID MESSAGE
    required = ['type', 'services', 'discovery']

    # DECODED KEYS
    keys = list(data.keys())
    
    # THE REQUEST KEYWORD FOR THE PAYLOAD TYPE
    keyword = 'request'
    
    # IF THE KEYSETS MATCH & THE TYPE IS A REQUEST
    if (required == keys and data['type'] == keyword):
        
        # CHECK MATCHES IN SERVICES & DISCOVERY PARAMS
        services_result = compare_services(data['services'], services)
        discovery_result = compare_discovery(data['discovery'], discovery_config)
        
        # COMBINE LISTS FOR VERIFICATION
        result = services_result + discovery_result
        
        # IF EVERYTHING MATCHED
        if (result.count(False) == 0):
            
            # SHOW MSG
            log('DISCOVERY REQUEST DETECTED')
            
            # ENCODE A JSON RESPONSE
            response = encode({
                'type': 'response',
                'source': payload,
                'device': device.hash
            })
            
            # SLEEP FOR 2 SECONDS
            time.sleep(2)

            # RESPOND TO REQUEST
            shh.post({
                'symKeyID': settings['whisper']['topic']['key'],
                'payload': blockchain.toHex(text=response),
                'topic': blockchain.toHex(text=settings['whisper']['topic']['name']),
                'sig': whisper_id,
                'powTarget': 2.5,
                'powTime': 2
            })

### START LISTENING FOR EVENTS

In [48]:
log('AWAITING EVENTS...\n')

[LAUNCHER] AWAITING EVENTS...



### CONTRACT EVENTS

In [49]:
update_event = device.event('middleware')

In [50]:
changes_event = device.event('changes')

In [51]:
assignment_event = device.event('assignment')

In [52]:
message_event = shh.newMessageFilter({
    'topic': blockchain.toHex(text=settings['whisper']['topic']['name']),
    'symKeyID': settings['whisper']['topic']['key']
})

### EVENT LOOP

In [53]:
try:
    while(True):
    
        # FETCH THE GLOBAL PARAMS
        global backlog
        global active
        global discoverable

        # ACTIVE STATUS EVENT
        for event in changes_event.get_new_entries():
            update_details(event)

        # UPDATE MIDDLEWARE EVENT
        for event in update_event.get_new_entries():
            update_middleware()

        # IF THE DEVICE IS SET TO ACTIVE
        if (active):

            # PERFORM TASKS IN BACKLOG
            for task in backlog:
                perform_task(task)
        
        # IF THE DEVICE IS DISCOVERABLE
        if (discoverable):
            
            # TRACK WHISPER REQUESTS
            for event in shh.getMessages(message_event):
                process_message(event)

# WHEN THE PROCESS IS KILLED...
except KeyboardInterrupt:
    print('\nThe process was manually stopped...')
    pass

[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED
[LAUNCHER] DISCOVERY REQUEST DETECTED

The process was manually stopped...


In [54]:
encode({
    'type': 'request',
    'services': ['foo'],
    'discovery': {
        'location': 'helsinki'
    }
})

'eyJ0eXBlIjogInJlcXVlc3QiLCAic2VydmljZXMiOiBbImZvbyJdLCAiZGlzY292ZXJ5IjogeyJsb2NhdGlvbiI6ICJoZWxzaW5raSJ9fQ=='

In [55]:
decode('eyJ0eXBlIjogInJlc3BvbnNlIiwgInNvdXJjZSI6ICJleUowZVhCbElqb2dJbkpsY1hWbGMzUWlMQ0FpYzJWeWRtbGpaWE1pT2lCYkltWnZieUpkTENBaVpHbHpZMjkyWlhKNUlqb2dleUpzYjJOaGRHbHZiaUk2SUNKb1pXeHphVzVyYVNKOWZRPT0iLCAiZGV2aWNlIjogImVhODkxNThhZGJlNTlhYTljNWU4MTQ3YzUwZjJlNWNjY2MzYTBkNTY1MjAyZDU4YWJjNTdkNWM4OTQ2Yjc4ZTcifQ==')

{'type': 'response',
 'source': 'eyJ0eXBlIjogInJlcXVlc3QiLCAic2VydmljZXMiOiBbImZvbyJdLCAiZGlzY292ZXJ5IjogeyJsb2NhdGlvbiI6ICJoZWxzaW5raSJ9fQ==',
 'device': 'ea89158adbe59aa9c5e8147c50f2e5cccc3a0d565202d58abc57d5c8946b78e7'}